In [19]:
import pandas as pd
import numpy as np
import json
from datetime import date
import urllib
#Вытащить данные из json по герою
# Kills 	Deaths 	Assists 	KDA 	Avg. KAL 	GPM 	XPM 	Last Hits 	Denies 	LVL 	HD 	TD 	HH 	GS
def get_data_for_Heroe(i):    
    data = [i.get('hero'), i.get('kills'), i.get('deaths'), i.get('assists'), i.get('kda'), i.get('avgKal'), i.get('gpm'), i.get('xpm'),
            i.get('lastHits'), i.get('denies'), i.get('level'), i.get('heroDamage'), i.get('towerDamage'),
            i.get('heroHealing'), i.get('goldSpent'),]
    return data

In [3]:
# # загрузить json с диска (НА РАБОТЫ)
# import codecs
# json_file = codecs.open( "basick.json")  
# dat = json.load(json_file)
# # создать словарь с данными по каждому герою из json 'dat'
# basick_perfomances={}
# for i in dat.get('data'):
#     data = get_data_for_Heroe(i)
#     basick_perfomances.update({i.get('hero'): data})

In [22]:
columns = ['hero', 'kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']

main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)

In [23]:
main[:3]

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,86.0,8.0,76.0,...,132851371.0,106573901.0,92423451.0,106809101.0,89871557.0,181716137.0,112377459.0,121052479.0,93616251.0,ESL One Genting 2017
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False,21.0,55.0,71.0,...,86726887.0,87196890.0,40547474.0,47434686.0,139876032.0,117281554.0,82327674.0,149486894.0,87012746.0,ESL One Genting 2017
2,2897155244,1483682366,2581813.0,16,2512249.0,26,False,79.0,63.0,28.0,...,180012313.0,116782914.0,152545459.0,102099826.0,87382579.0,103735745.0,38628747.0,86700461.0,86725175.0,ESL One Genting 2017


In [40]:
# два ДФ для героев по патчам  в целом и для героев по каждой команде и создать в низ столбец с id матча
df_basick_peromances_heroes = pd.DataFrame()
df_basick_peromances_heroes_teams = pd.DataFrame()
df_basick_peromances_heroes['match_id'] = main['match_id']
df_basick_peromances_heroes_teams['match_id'] = main['match_id']

# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv')
main = main[:1]

# создать файлы с KDA героев по каждой дате (с 2017 года используется только патч 7,00+)
year = 2016
for month in range (12, 13):
    for day in range (11, 32):
         # создать правильную ссылку для поиска по всем героям 
        url_heroes =('http://www.datdota.com/api/heroes/performances?patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&'+ 
               "patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&".format(
               day, month, year) + 
               "duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1") 
        # выгрзить json с предыдущей ссылки
        sauce = urllib.request.Request(url_heroes, headers={'User-agent': 'Mozilla/5.0'} )
        dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
        df = pd.DataFrame(dat, columns=columns)
        
        # запись фала на диск
        df.to_csv('../tabel with heroes DATDOTA (kda)/{}-{}-{}.csv'.format(year, month, day))




# for index in main.index:
#     # взять одну строчку
#     one_match = main.loc[[index]]
#     # достать дату матча
#     date_match = date.fromtimestamp(one_match['start_time'][index])
    
#     # id команды из данных по матчу
#     id_team_radiant = one_match['radiant_team_id'][index]
#     id_team_dire = one_match['dire_team_id'][index]
    
#     # создать правильную ссылку для поиска по всем героям 
#     url_heroes =('http://www.datdota.com/api/heroes/performances?patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&'+ 
#            "patch=7.03&patch=7.02&patch=7.01&patch=7.00&winner=either&after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&".format(
#            date_match.day, date_match.month, date_match.year) + 
#            "duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1") 
#     # выгрзить json с предыдущей ссылки
#     sauce = urllib.request.Request(url_heroes, headers={'User-agent': 'Mozilla/5.0'} )
#     dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
#     df = pd.DataFrame(dat, columns=columns)
    
    
    
    
#     basick_perfomances = {}
#     for i in dat:
#         data = get_data_for_Heroe(i)
#         basick_perfomances.update({i.get('hero'): data})


In [36]:
df

,hero,kills,deaths,assists,kda,avgKal,gpm,xpm,lastHits,denies,level,heroDamage,towerDamage,heroHealing,goldSpent
0,28,1.783784,5.621622,14.702703,2.932692,3.617131,308.567568,369.189189,65.594595,3.189189,16.918919,6091.837838,252.864865,49.756757,8843.918919
1,9,4.230769,4.730769,12.576923,3.552846,4.625018,410.653846,400.692308,148.576923,2.307692,17.769231,14134.615385,1038.076923,554.807692,12585.000000
2,84,2.230769,5.653846,12.807692,2.659864,2.768539,278.153846,312.576923,36.653846,2.423077,15.884615,6960.692308,309.076923,715.807692,8900.000000
3,8,5.360000,3.520000,8.360000,3.897727,4.361571,505.880000,444.160000,221.920000,14.920000,18.160000,14102.080000,3396.160000,0.000000,15825.200000
4,65,3.833333,5.125000,14.000000,3.479675,4.649418,354.875000,389.000000,87.958333,2.666667,16.708333,11393.458333,321.541667,0.000000,9674.791667
5,48,6.434783,3.173913,8.521739,4.712329,7.105314,608.000000,521.000000,245.478261,11.173913,18.913043,15998.086957,6384.956522,0.000000,16658.913043
6,79,2.428571,5.523810,11.190476,2.465517,2.702234,245.952381,244.000000,34.428571,1.428571,12.952381,6598.428571,906.095238,356.142857,7362.142857
7,87,2.600000,6.650000,11.600000,2.135338,2.516822,245.800000,265.700000,23.600000,1.900000,14.100000,6908.450000,201.700000,238.550000,8023.250000
8,76,8.450000,4.850000,8.250000,3.443299,4.072944,489.350000,506.100000,196.600000,11.450000,19.850000,19226.250000,1862.200000,0.000000,15333.000000
9,51,4.526316,6.052632,11.263158,2.608696,3.461028,329.052632,331.684211,70.157895,18.000000,15.684211,15176.157895,246.052632,309.105263,8970.263158


In [ ]:
    # создать правильную ссылку для посика героев по команде radiant
    url_heroes_team_radiant = ('http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&'.format(
                        id_team_radiant) + 'patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&winner=either&' + 
                       'after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&duration-value-from=0&'.format(
                       date_match.day, date_match.month, date_match.year) + 
                       'duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1')
    # создать правильную ссылку для посика героев по команде dire
    url_heroes_team_dire = ('http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&'.format(
                        id_team_dire) + 'patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&winner=either&' + 
                       'after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&duration-value-from=0&'.format(
                       date_match.day, date_match.month, date_match.year) + 
                       'duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1')